In [ ]:
# Let's try the DeepLifting MLP on the Bukin function
# we already saw that it had a great solution for the
# Ackley function

In [ ]:
import time
import sys

# Append the path for this notebook
sys.path.append("/Users/ryandevera/data-science/umn_environments/Deeplifting")

import matplotlib.pyplot as plt
import numpy as np
import torch
from pygranso.private.getNvar import getNvarTorch
from pygranso.pygranso import pygranso
from pygranso.pygransoStruct import pygransoStruct

from deeplifting.models import DeepliftingMLP

In [ ]:
# Let's create the objective
def bukin_n6(x):
    return 100.0 * torch.sqrt(torch.abs(x[1] - 0.01 * x[0] ** 2)) + 0.01 * torch.abs(
        x[0] + 10.0
    )


# Check the right value is returned at known local minima
x = torch.tensor([-10.0, 1.0])
print(bukin_n6(x))

In [ ]:
# Get the device type
# For now we will just utilize cpu
device = torch.device('cpu')


# Create the combined function
def bukin_fn(model):
    """
    Combined function we will take the model as input
    """
    # Remember we are letting the input be
    # variable
    x = model(inputs=None)

    # Objective function
    f = bukin_n6(x)

    # Inequality constraint
    ci = pygransoStruct()
    ci.c1 = -15.0 - x[0]
    ci.c2 = x[0] + 5.0
    ci.c3 = -3.0 - x[1]
    ci.c4 = x[1] - 3.0

    # equality constraint
    ce = None

    return f, ci, ce


# Define the model
model = DeepliftingMLP(input_size=100, layer_sizes=(256, 512, 512, 256), output_size=2)
model = model.to(device=device, dtype=torch.double)
nvar = getNvarTorch(model.parameters())
print(nvar)

In [ ]:
torch.manual_seed(0)
# Setup a pygransoStruct for the algorithm
# options
opts = pygransoStruct()

# With random variables between -32, 32
x0 = (
    torch.nn.utils.parameters_to_vector(model.parameters())
    .detach()
    .reshape(nvar, 1)
    .to(device=device, dtype=torch.double)
)

opts.x0 = x0
opts.torch_device = device
opts.print_frequency = 10
opts.limited_mem_size = 50
opts.stat_l2_model = False
opts.double_precision = True
opts.viol_ineq_tol = 1e-6
opts.opt_tol = 1e-6

comb_fn = lambda model: bukin_fn(model)

# Run the main algorithm
start = time.time()
soln = pygranso(var_spec=model, combined_fn=comb_fn, user_opts=opts)
end = time.time()

print(f"Starting point for problem = {x0}")
print("\n")
print("Total Wall Time: {}s".format(end - start))
print(soln.final.x, soln.final.f)
print("\n")

In [ ]:
x = model(inputs=None)
print(x, bukin_n6(x))